In [140]:
# 导入模块
%matplotlib inline
import torch
import numpy as np
import pylab as pl
from torch import nn
import re

torch.manual_seed(1)
np.random.seed(1)

In [201]:
# 训练一个基于ERNN神经网络来作诗

## 读入用GloVe处理得到的文字 embeddings，以及句子数据。
import codecs

word_emb_dim = input_size = 1
i2w = {0:''}
w2i = {'':0}

word_emb_dim = 128

with codecs.open('data/word_embeddings_manyun-mix_128.txt', mode='r', encoding='utf-8') as f:
    lines = f.readlines()
    n_words = len(lines)+1
    word_embeddings = torch.nn.Embedding(n_words, word_emb_dim)
    for i in range(1, n_words):
        line = lines[i-1].split(' ')
        i2w[i] = line[0]
        w2i[line[0]] = i
        word_embeddings.weight[i] = torch.from_numpy(np.array(line[1:],dtype=np.float32))

max_line_length = 50
poems = []
with codecs.open('data/manyun-mix.txt', mode='r', encoding='utf-8') as f:
    for poem in f:
        poem = re.sub('\s','',poem)
        if ':' in poem:
            poem = poem.split(':')[-1]
        poem = 'S'+poem+'E'
        if len(poem) < 10 or len(poem) > max_line_length or '(' in poem or u'（' in poem or u'《' in poem or '-' in poem or '_' in poem:
            continue
#        poem = re.split(u'[。；.?？]', poem)
#        for s in poem:
#            if len(s)>3:
#                s += u'。'
        poems.append(map(w2i.get, poem))

n_poems = len(poems)

print( 'Data summary:\n\n number of poems: {}\n number of words: {}\n'.format(n_poems, n_words))
print('Poem examples:\n\n'+'\n'.join([''.join(map(i2w.get, x)) for x in poems[:10]]))
    

Data summary:

 number of poems: 220
 number of words: 1756

Poem examples:

长空雁，霜晨月。投笔怀牧之，飞书笑鲁连。红尘踏马惯烟雪，孤村坐卧赏寒绝。E
寄琴剑，托凤笺。素肴酬知己，行杯梦真玄。水卢文艺遣高情，古今风物共流年。E
平生何所寄？天地一孤篷。郁纡且行游，迟复尘景中。E
星汉奔岩屿，惊涛卷曈虹。翕趿隐烟色，长桥海岛空。E
百年如云梦，逆旅何匆匆。吟坐忘知闻，拈花鉴溟濛。E
道心不外求，日影养虚冲。观风遣剑意，抱朴任穷通。E
千古一杯清，卧剑亦何如？云雁有芳信，谈笑未成书。E
故国弛山色，春华因才逐。北庭惜玉折，积风待岁除。E
俯仰苍茫间，太虚应有诸。值此吟月夜，借居怀纡余。E
心斋即坛醮，守道安违俗。江湖得意气，狂歌岂踟躇。E


In [258]:
# 定义一个函数，随机返回一个 mini batch，用于训练，由于每一首诗歌的长度不同，我们此处规定每个batch只有一首诗。这样，就可以生成长度可变的诗歌。
def get_batch(batch_size=2):
    batch_raw = [poems[i][:] for i in np.random.randint(0, n_poems, batch_size)]
    max_length = max(map(len, batch_raw))
    for i in range(len(batch_raw)):
        for j in range(len(batch_raw[i]),max_length):
            batch_raw[i].append(0)
    batch_raw = torch.LongTensor(batch_raw).detach().unsqueeze(2).transpose(0,1)
    x = batch_raw[:-1].type(torch.float32)
    y = batch_raw[1:]
    return x, y

def idx2emb(x):
    return word_embeddings(x.type(torch.long)).squeeze(2).detach()
    

# 定义一个函数，输入一个 batch 返回句子
def batch2sent(batch):
    S = []
    batch = batch.type(torch.int32).detach()
    seq_length, batch_size, emb_size = batch.size()
    for i in range(batch_size):
        S.append(''.join(map(i2w.get, batch[:,i,:].view(-1).tolist())))
    return u'\n'.join(S)

x, y = get_batch(1)
print(batch2sent(x))
print(batch2sent(y))

# 定义一个生成器
class Generator(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, n_layers=2, activation=None):
        super(Generator, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.activation = activation
        self.rnn = nn.LSTM(self.input_size, self.hidden_size, num_layers=self.n_layers, dropout=0.01)
        self.output = nn.Linear(self.hidden_size,self.output_size)
        self.softmax = torch.nn.LogSoftmax(dim=-1)
    def init_h(self):
        return (torch.randn(self.n_layers, self.batch_size, self.hidden_size),torch.randn(self.n_layers, self.batch_size, self.hidden_size))
    def forward(self, x, h0=None):
        self.seq_length, self.batch_size, self.input_size = x.size()
        if h0 is None:
            h0 = self.init_h()
#            x0 = torch.FloatTensor([w2i['S']]).view(1,1,-1).detach()
#            x0 = idx2emb(x0)
#            y0, h0 = self.rnn(x0,h0)
        y, ht = self.rnn(x,h0)
#        y = torch.cat((y0,y),dim=0)
        y = y.view(-1,self.hidden_size)
        y = self.output(y)
        y = y.view(self.seq_length,self.batch_size,self.output_size)
        y = self.softmax(y)
        return y, ht

def poem_gen(model, w=None, cr=1e-1):
    with torch.no_grad():
        if not w in w2i or w is None:
            idx = np.random.randint(1,n_words)
            w = i2w[idx]
        else:
            idx = w2i[w]
        ht = None
        x0 = torch.FloatTensor([w2i['S']]).view(1,1,-1).detach()
        x0 = idx2emb(x0)
        y, ht = model(x0, ht)
        x = torch.LongTensor([w2i[w]]).view(1,1,-1).detach()
        x = idx2emb(x)
        s = []
        s.append(w)
        for t in range(max_line_length):
            y, ht = model(x, ht)
            not_done = True
            cnt = 0
            while not_done and cnt <50:
                k = min([1+np.random.binomial(3,0.5), y.size(-1)-1])
                x = torch.topk(y, k, dim=-1)[1].detach()
                x = x[:,:,min([np.random.geometric(0.3), k-1])].unsqueeze(2)
#                x = torch.argmax(y,dim=-1,keepdim=True)
                cnt += 1
                w = batch2sent(x)
                if s.count(w)<1: not_done = False
            if w == 'E':
                break
            s.append(w)
            x = idx2emb(x)
        return u''.join(s)
    
    
# 训练一个简单的 RNN 模型以生成诗歌

input_size = word_emb_dim
hidden_size = 128
output_size = n_words
activation = torch.relu

model = Generator(input_size, output_size, hidden_size, n_layers=2, activation=activation)


从来湖上胜人间，远爱浮云独自还。孤月空天见心地，寥寥一水镜中山。
来湖上胜人间，远爱浮云独自还。孤月空天见心地，寥寥一水镜中山。E


In [259]:
lr = 1e-3
n_epochs = 10000
last_epoch = -1
disp_interval = 50
batch_size = 1

loss_func = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

torch.manual_seed(1)
np.random.seed(1)

def lr_lambda(epoch):
    return 0.99**(epoch/50.0)

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=last_epoch)

model.load_state_dict(torch.load('saves/model-manyun.pt'))

Loss = []
for epoch in range(n_epochs):
    model.zero_grad()
    x_obs, y_obs = get_batch(batch_size=batch_size)
    x_obs = idx2emb(x_obs)
    y_pred, ht = model(x_obs)
    y1 = torch.argmax(y_pred.detach(),-1,keepdim=True).detach()#[:,:1,:]
    y2 = y_obs.detach()#[:,:1,:]
    y_pred = y_pred.view(-1,output_size)
    y_obs = y_obs.contiguous().view(-1)
    loss = loss_func(y_pred,y_obs)
    loss.backward()
    Loss.append(loss.tolist())
    optimizer.step()
    scheduler.step()
    if epoch % disp_interval == 0:
        print(u'Epoch{}, Loss{}\nPred:\n{}\nObs:\n{}\nRnd:\n{}\n'.format(epoch,loss.tolist(), batch2sent(y1), batch2sent(y2),poem_gen(model)))
        torch.save(model.state_dict(),'saves/model-manyun.pt')
window_size = 50
avg_losses = np.array(Loss)[:len(Loss)//50 *50].reshape([-1,window_size]).mean(1)
pl.plot(np.arange(0,len(Loss)//50 *50,window_size), avg_losses,'r-')
pl.xlabel('Time')
pl.ylabel('Loss')
pl.yscale('log')


Epoch0, Loss0.0010306944605
Pred:
翩公子，在陌之桑。剑胆琴心，气宇轩昂。E
Obs:
翩公子，在陌之桑。剑胆琴心，气宇轩昂。E
Rnd:
信颠踣故今无过，禅心正欲甚慰天冷。况一春此忧尽人地人

Epoch50, Loss0.0782466679811
Pred:
阳台昙花三五成蕾，似欲娇绽，余心甚慰，常与之对坐倾谈。E
Obs:
阳台昙花三五成蕾，似欲娇绽，余心甚慰，常与之对坐倾谈。E
Rnd:
婺小削奇秀人秋思意，禅资去时迟云北子归来月。知堪有欢见尽心

Epoch100, Loss0.175071612
Pred:
假独拥酒醒，纵有兴、怕逞余杯。E
Obs:
竟独拥酒醒，纵有兴、怕逞余杯。E
Rnd:
生分已在心名，今日无我须放。一朝天柳色落客雪客人

Epoch150, Loss0.100216180086
Pred:
甚少在人群之中提及妳E
Obs:
甚少在人群之中提及妳E
Rnd:
鲁颠行人路远寒。山情海、秋归，芳我亦天无中苑云湘枝长谁忆冷尽。

Epoch200, Loss0.206890508533
Pred:
日趣不已，东风吹绿蘋。欲看梅市雪，知赏柳家春。别意倾吴醑，芳声动越人。山阴三月会，内史得嘉宾。E
Obs:
舸趣不已，东风吹绿蘋。欲看梅市雪，知赏柳家春。别意倾吴醑，芳声动越人。山阴三月会，内史得嘉宾。E
Rnd:
共风子出人在心，细舟今留家石情

Epoch250, Loss0.228725373745
Pred:
欲性情奇，初生玉树枝。人曾天上见，名向月中知。我识婴儿意，何须待佩觿。E
Obs:
子性情奇，初生玉树枝。人曾天上见，名向月中知。我识婴儿意，何须待佩觿。E
Rnd:
边马后生，秋古一何天行云空烟枝落。离上摇子应远情心名知我与柏年

Epoch300, Loss0.570518374443
Pred:
吹未长已暮雨难将息。叶底揽花痕，并蒂瘦、娇无欢意。E
Obs:
情未已，暮雨难将息。叶底揽花痕，并蒂瘦、娇无欢意。E
Rnd:
几恨酣游春殿，来不撩雨霜上年。一翠禅阳遥隐夜人雪雪江云晚去雁心寂我



KeyboardInterrupt: 

In [273]:
print poem_gen(model,u'荷')

荷池台上救散，太日梅雪谁？无意行强欢尽。一朝万里事客雪。知看多禅及心公


In [237]:
x.size(2)

1

In [ ]:
x = torch.randn(2,3,4)

In [6]:
y_obs.size()

torch.Size([25, 5, 1])

In [ ]:
torch.transpose(input,1,0).size()

In [ ]:
torch.topk(torch.randn(1,1,100),1,-1)[1].shape

In [64]:
np.random.binomial(6,0.5)

3

In [ ]:
x.size()

In [ ]:
a[:]

In [ ]:
x.size()

In [ ]:
y.size()

In [118]:
x = torch.randint(0,10,[3,4])
print(x)

tensor([[ 6.,  6.,  4.,  0.],
        [ 4.,  0.,  8.,  2.],
        [ 0.,  8.,  9.,  2.]])


In [109]:
idx2emb(torch.LongTensor([[[ 236]]])
).size()

torch.Size([1, 1, 128])

In [121]:
torch.topk(x,3,dim=-1)[1]

tensor([[ 0,  1,  2],
        [ 2,  0,  3],
        [ 2,  1,  3]])